<a href="https://colab.research.google.com/github/tjturnage/colab-dash/blob/main/xmacis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2><b>Step 1</b></h2>
<h3>Install the packages you need. You can ignore the comments that appears as long as there are no error messages.</h3>

In [9]:
!pip install jupyter-dash -q
!pip install dash-bootstrap-components
import json
import requests
from datetime import datetime
import numpy as np
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html, Output, Input, no_update
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h2><b>Step 2</b></h2>
<h3>Determine a station you want to get data for (note: long term threaded datasets end with "thr").<br>Detailed information about ACIS Web Services <a href="https://www.rcc-acis.org/docs_webservices.html" target="_blank">HERE</a></h3>


In [3]:

station_name = "GRRthr" #@param ["GRRthr"] {allow-input: true}

try:
    params = {"sids": station_name, "elems": "maxt", "meta": ["name","valid_daterange"]}
    req = requests.post(meta_url,json=params)
    data = req.json()
    meta = data['meta'][0]
    name = meta['name']
    print(f'The default name of the station is: {name}.')
except Exception:
  print("Station Not Found! Check your entry and try again.")



The default name of the station is: Grand Rapids Area.


In [4]:
#@markdown If you don't like the default station name enter a new name below that you'd rather see on the graph. Otherwise ensure field below is blank.
new_name= "Grand Rapids" #@param {type:"string"}

<h2><b>Step 3</b></h2>
<h3>Run the code below to retrieve data</h3>


In [10]:


mon = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

parms_url = "http://data.rcc-acis.org/StnData?params="
meta_url = "http://data.rcc-acis.org/StnMeta"

class XMACIS():
    def __init__(self,station):

        self.station = station

        #grab metadata related to station's Period Of Record (POR)
        self.por_metadata = self.get_por_metadata()
        self.df = self.get_all_por_data()
        self.melted = pd.melt(self.df, id_vars=['year','month','day'], value_vars=['mint', 'maxt', 'avgt', 'maxt_n','mint_n','avgt_n'], value_name='temperature')
        self.melted = self.melted.astype({"variable": 'string'}, errors='ignore')


    def get_por_metadata(self):
        """
        Returns list of strings: ['YYYY-mm-dd','YYYY-mm-dd']
        Refers to first/last dates ofperiod of record
        """
        params = {"sids": self.station, "elems": "maxt", "meta": ["name","valid_daterange"]}
        req = requests.post(meta_url,json=params)
        data = req.json()
        meta = data['meta'][0]
        self.full_name = meta['name']
        try:
          self.full_name = new_name
        except:
          pass
        self.por_start_date_str = meta['valid_daterange'][0][0]
        self.por_end_date_str = meta['valid_daterange'][0][1]
        self.por_start_date_obj = datetime.strptime(self.por_start_date_str, '%Y-%m-%d')
        self.por_end_date_obj = datetime.strptime(self.por_end_date_str, '%Y-%m-%d')
        self.por_start_year = self.por_start_date_obj.year
        self.por_end_year = self.por_end_date_obj.year
        self.por_year_range = self.por_end_year - self.por_start_year + 1
        self.por_year_index = pd.date_range(start=f'1/1/{self.por_start_year}', periods=self.por_year_range, freq='Y')
        self.por_label = f'{self.por_start_year} - {self.por_end_year}'
        return

    def get_all_por_data(self):
        parms = {
        "sid":self.station,
        "sdate":self.por_start_date_str,
        "edate":self.por_end_date_str,
        "elems":[
            {"name":"maxt","interval":"dly","duration":"dly"},
            {"name":"mint","interval":"dly","duration":"dly"},
            {"name":"avgt","interval":"dly","duration":"dly"},
            {"name":"maxt","interval":"dly","duration":"dly","normal":"1"},
            {"name":"mint","interval":"dly","duration":"dly","normal":"1"},
            {"name":"avgt","interval":"dly","duration":"dly","normal":"1"},
            ]}

        req = requests.post(parms_url,json=parms)
        data = req.json()
        df_init = pd.json_normalize(data)
        df_dp = df_init["data"].apply(pd.Series).T
        df = df_dp[0].apply(pd.Series)
        df.columns = ['dts','maxt','mint','avgt','maxt_n','mint_n','avgt_n']
        df['date'] = pd.to_datetime(df['dts'], errors='coerce')
        #df['date'] = pd.DatetimeIndex(df['dts'])
        df.set_index('date', inplace=True)
        df['year'] = df.index.year
        df['month'] = df.index.month
        df['day'] = df.index.day
        df = df.astype({"dts":'string', "year":'int', "month":'int', "day":'int', "maxt":'int', "mint":'int', "avgt":'float', "maxt_n":'float', "mint_n":'float', "avgt_n":'float'}, errors='ignore')
        return df


test = XMACIS(station_name)

dff = test.melted
por_start = test.por_start_year
por_end = test.por_end_year
yrs = list(range(por_start,por_end+1))
dff = dff.astype({"variable": 'string'}, errors='ignore')
dff['mon'] = dff['month'].map(mon)

D = test.df
# remove leap days ... too much trouble
D = D[~((D.index.month == 2) & (D.index.day == 29))]
#output = test.df.to_csv(index=False,header=True)
#with open('climate-data.csv', 'w') as outfile:
#  for lines in output:
#    outfile.write(lines)

<h2><b>Step 4</b></h2>
<h3>Run code below to plot data. 
Once it completes, scroll down to see interactive graph.<br><i>(this might take a while to render at first)</i></h3>


In [8]:
months = list(np.arange(1,13))
dates = list(np.arange(1,32))

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])

app.layout = html.Div([
    dbc.Row(html.H6(' ')),
    dbc.Row([
        dbc.Col([
            html.H6('Graph Type'),
            dcc.Dropdown(['violin'], value='violin', id='graph_type',searchable=False)
        ]),
        dbc.Col([
            html.H6('Start Date'),
            dcc.Dropdown(yrs, value=por_start, id='base_start',searchable=False)
        ]),
        dbc.Col([
            html.H6('Select end year'),
            dcc.Dropdown(yrs, value=por_end, id='base_end',searchable=False)
        ]),
        dbc.Col([
            html.H6('Select element to investigate'),
            dcc.Dropdown(
                options=[
                    {'label': 'Min Temp', 'value': 'mint'},
                    {'label': 'Max Temp', 'value': 'maxt'},
                    {'label': 'Avg Temp', 'value': 'avgt'}#,
                    ],
                    value='maxt',id="element", searchable=False)
        ]),
        dbc.Col([
            html.H6('Year to compare'),
            dcc.Dropdown(yrs, value=2015, id='single_year',searchable=False)
        ])
    ]),
    dbc.Row(html.H6(' ')),
    dbc.Row(dcc.Graph(id="graphing"))
])


@app.callback(
    Output("graphing", "figure"),
    Input("graph_type", "value"),
    Input("base_start", "value"),
    Input("base_end", "value"),
    Input("element", "value"),
    Input("single_year", "value"))
def update_violin_chart(graph_type,base_start,base_end,element,single_year):
  df = dff[(dff['variable'] == element)]
  nice_name = 'Minimum Temperature'
  col = 'blue'
  if element == 'maxt':
    nice_name = 'Maximum Temperature'
    col = 'red'
  if element == 'avgt':
    nice_name = 'Average Temperature'
    col = 'orange'
  df_base = df[(df['year'] >= base_start) & (df['year'] <= base_end)]
  df_year = df[df['year'] == single_year]

  fig = go.Figure()
  fig.add_trace(go.Violin(x=df_base['mon'], y=df_base['temperature'],
                          legendgroup='Yes', scalegroup='Yes', name=f'{base_start}-{base_end}',
                          side='negative',
                          line_color='gray')
              )
  fig.add_trace(go.Violin(x=df_year['mon'],
                          y=df_year['temperature'],
                          legendgroup='No', scalegroup='No', name=f'{single_year}',
                          side='positive',
                          line_color=col)
              )
  fig.update_traces(meanline_visible=True)
  fig.update_layout(violingap=0, violinmode='overlay')
  r = [-30,120]
  fig.update_yaxes(range = r)
  fig.update_layout(autotypenumbers='convert types')
  fig.update_layout(title=f'{test.full_name} - {nice_name}',
                    legend_title='',
                    yaxis_title='Temperature  (F)')
  fig.update_layout(margin=dict(l=20, r=10, t=60, b=20))

  return fig

app.run_server(mode='inline', port=8030)

Dash is running on http://127.0.0.1:8030/



INFO:dash.dash:Dash is running on http://127.0.0.1:8030/



<IPython.core.display.Javascript object>

<h1><b>All code below is in development. Ignore for now!</b></h1>

In [ ]:
months = list(np.arange(1,13))
dates = list(np.arange(1,32))


#app.layout = html.Div([
#    dbc.Row(html.H6(' ')),
#    dbc.Row([
#        dbc.Col([
#            html.H6('Select year'),
#            dcc.Dropdown(yrs, value=2015, id='year',searchable=False)
#        ]),
#        dbc.Col([
#            html.H6('Select month'),
#            dcc.Dropdown(months, value=6, id='month',searchable=False)
#        ]),
#        dbc.Col([
#            html.H6('Select element'),
#            dcc.Dropdown(
#                options=[
#                    {'label': 'Min Temp', 'value': 'mint'},
#                    {'label': 'Max Temp', 'value': 'maxt'},
#                    {'label': 'Avg Temp', 'value': 'avgt'}#,
#                    ],
#                    value='maxt',id="heatmap_element", searchable=False)
#        ]),
#    ]),
#    dbc.Row(html.H6(' ')),
#    dbc.Row(dcc.Graph(id="heatmap"))
#])


#@app.callback(
#    Output("graphing", "figure"),
#    Input("year", "value"),
#    #Input("month", "value"),
#    Input("heatmap_element", "value"))
year=2014
month = 2
element = 'maxt'
def update_heatmap(year,month,element):
  df = dff[(dff['variable'] == element)]
  nice_name = 'Minimum Temperature'
  col = 'blue'
  if element == 'maxt':
    nice_name = 'Maximum Temperature'
    col = 'red'
  if element == 'avgt':
    nice_name = 'Average Temperature'
    col = 'orange'
  df_heatmap = df[(df['year'] >= year) & (df.index.month == month)]


  #fig = go.Figure(data=go.Heatmap(
          #x = months,
          #y = dates,
          #z = df_heatmap[element],
          #type = 'heatmap',
          #colorscale = 'Viridis'))

  fig = go.Figure(data=go.Heatmap(
          x = [1,2,3],
          y = [4,5,6],
          z = [[9,8,7],
               [6,5,4],
               [3,2,1]],
          type = 'heatmap',
          colorscale = 'Viridis'))
  

  #fig.update_traces(meanline_visible=True)
  #fig.update_layout(violingap=0, violinmode='overlay')
  #r = [-30,120]
  #fig.update_yaxes(range = r)
  #fig.update_layout(autotypenumbers='convert types')
  #fig.update_layout(title=f'{test.full_name} - {nice_name}',
  #                  legend_title='',
  #                  yaxis_title='Temperature  (F)')
  #fig.update_layout(margin=dict(l=20, r=10, t=60, b=20))
  return fig

#app.run_server(mode='inline', port=8030)
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime,timedelta

day_num = []
doy_strings = []
for doy in range(0,365):
  base = datetime(1999,1,1)
  daymonth_datetime = base + timedelta(days=int(doy))
  doy_strings.append(daymonth_datetime.strftime('%b-%d'))

x_label = []
for ii, d in enumerate(doy_strings):
  if '01' in d:
    day_num.append(ii)
    x_label.append(d)

df = test.df
df_base = df['2010-01-01' : '2020-12-31']
#print(df_base)



fig1 = px.scatter(x=df.index.dayofyear, y=df['maxt'], color=df.index.year)
fig2 = px.scatter(x=df_base.index.dayofyear, y=df_base['maxt'])
fig2.update_traces(marker=dict(color='black'))
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.update_layout(autotypenumbers='convert types')
fig3.show()
#fig.add_scatter(x=df_base.index.dayofyear, y=df_base['maxt_n'], linestyle=':', color='r', linewidth=3, alpha=0.75) # plot normals

